In [268]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import tensorflow as tf

In [269]:
test_data = pd.read_csv("test.csv")
train_data = pd.read_csv("train.csv")
y_train = train_data["Survived"]
del train_data["Survived"]
df = pd.concat([train_data,test_data],axis=0)

In [270]:
df["Name"]= df["Name"].str.split(",", n = 1, expand = True)[1]
df["Name"]= df["Name"].str.split(".", n = 1, expand = True)[0]
df["Name"].replace([" Mr"," Miss"," Mrs"," Master"," Rev"," Dr" ," Col"," Mlle"," Major"," Ms"," Lady", " Sir"," Mme",
                   " Don"," Capt"," the Countess"," Jonkheer", " Dona"],
                   ["Mr","Master","Mrs","Master","Rev","Dr" ,"Major","Master","Major","Mrs","Mrs","Mr","Miss","Major","Major",
                    "Major","Major", "Dr"], inplace = True)

df["family member"] = df["SibSp"] + df["Parch"] + 1
df["Fare"] = df["Fare"].fillna(df["Fare"].mean())
df["Name"].value_counts()

Mr        758
Master    323
Mrs       200
Major      10
Dr          9
Rev         8
Miss        1
Name: Name, dtype: int64

In [271]:
df["Fare_per_person"] = df["Fare"]/df["family member"]
del df["PassengerId"]
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,family member,Fare_per_person
0,3,Mr,male,22.0,1,0,A/5 21171,7.2500,NaN,S,2,3.62500
1,1,Mrs,female,38.0,1,0,PC 17599,71.2833,C85,C,2,35.64165
2,3,Master,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1,7.92500
3,1,Mrs,female,35.0,1,0,113803,53.1000,C123,S,2,26.55000
4,3,Mr,male,35.0,0,0,373450,8.0500,NaN,S,1,8.05000


In [272]:
column = df.columns
typ = df["Ticket"].dtype
column = [col for col in column if not df[col].dtype== typ]
int_df = df[np.array(column)].copy()
int_df.head()

,Pclass,Age,SibSp,Parch,Fare,family member,Fare_per_person
0,3,22.0,1,0,7.2500,2,3.62500
1,1,38.0,1,0,71.2833,2,35.64165
2,3,26.0,0,0,7.9250,1,7.92500
3,1,35.0,1,0,53.1000,2,26.55000
4,3,35.0,0,0,8.0500,1,8.05000


In [273]:
for i in range(int_df.shape[0]):
    dataset = int_df.iloc[i:i+1,:]
    if(dataset["Fare_per_person"].values[0] <= 10 and dataset["Fare_per_person"].values[0] > 0):
        dataset["Fare_per_person"].values[0] = 1
        int_df.iloc[i:i+1,:] = dataset
    if(dataset["Fare_per_person"].values[0] > 10 and dataset["Fare_per_person"].values[0] <= 50):
        dataset["Fare_per_person"].values[0] = 2
        int_df.iloc[i:i+1,:] = dataset
    if(dataset["Fare_per_person"].values[0] > 50 and dataset["Fare_per_person"].values[0] <= 200):
        dataset["Fare_per_person"].values[0] = 3
        int_df.iloc[i:i+1,:] = dataset
    if(dataset["Fare_per_person"].values[0] > 200):
        dataset["Fare_per_person"].values[0] = 4
        int_df.iloc[i:i+1,:] = dataset

In [274]:
int_df["Age"] = int_df["Age"].fillna(df["Age"].mean())
int_df.isnull().sum()

Pclass             0
Age                0
SibSp              0
Parch              0
Fare               0
family member      0
Fare_per_person    0
dtype: int64

In [275]:
for i in range(int_df.shape[0]):
    dataset = int_df.iloc[i:i+1,:]
    if(dataset["Age"].values[0] <= 11):
        dataset["Age"].values[0] = 0
        int_df.iloc[i:i+1,:] = dataset
    if(dataset["Age"].values[0] > 11 and dataset["Age"].values[0] < 18):
        dataset["Age"].values[0] = 1
        int_df.iloc[i:i+1,:] = dataset
    if(dataset["Age"].values[0] >= 18 and dataset["Age"].values[0] < 22):
        dataset["Age"].values[0] = 2
        int_df.iloc[i:i+1,:] = dataset
    if(dataset["Age"].values[0] >= 22 and dataset["Age"].values[0] < 27):
        dataset["Age"].values[0] = 3
        int_df.iloc[i:i+1,:] = dataset
    if(dataset["Age"].values[0] >= 27 and dataset["Age"].values[0] < 33):
        dataset["Age"].values[0] = 4
        int_df.iloc[i:i+1,:] = dataset
    if(dataset["Age"].values[0] >= 33 and dataset["Age"].values[0] < 40):
        dataset["Age"].values[0] = 5
        int_df.iloc[i:i+1,:] = dataset
    if(dataset["Age"].values[0] >= 40 and dataset["Age"].values[0] < 46):
        dataset["Age"].values[0] = 6
        int_df.iloc[i:i+1,:] = dataset
    if(dataset["Age"].values[0] >= 46):
        dataset["Age"].values[0] = 7
        int_df.iloc[i:i+1,:] = dataset

In [276]:
col_str = [col for col in df.columns if not col in column]
col_str

['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']

In [277]:
str_df = df[np.array(col_str)].copy()
str_df.head()

,Name,Sex,Ticket,Cabin,Embarked
0,Mr,male,A/5 21171,NaN,S
1,Mrs,female,PC 17599,C85,C
2,Master,female,STON/O2. 3101282,NaN,S
3,Mrs,female,113803,C123,S
4,Mr,male,373450,NaN,S


In [278]:
cabin = str_df["Cabin"]
# cabin["Cabin"] = cabin["Cabin"].fillna(0)
# cabin[cabin["Cabin"] != 0] = 1
cabin = pd.DataFrame(cabin)

In [279]:
cabin = cabin.fillna(0)
cabin[cabin["Cabin"] != 0] = 1

In [280]:
int_df = pd.concat([int_df,cabin],axis=1)
del str_df["Cabin"]
del str_df["Ticket"]
str_df

,Name,Sex,Embarked
0,Mr,male,S
1,Mrs,female,C
2,Master,female,S
3,Mrs,female,S
4,Mr,male,S
...,...,...,...
413,Mr,male,S
414,Dr,female,C
415,Mr,male,S
416,Mr,male,S


In [281]:
str_columns = str_df.columns
for col in str_columns:
    vect = pd.get_dummies(df[col], prefix=col+"_")
    del str_df[col]
    str_df = pd.concat([str_df,vect], axis = 1)
    
str_df.shape

(1309, 12)

In [282]:
int_df.head()
int_df["Pclass_age"] = int_df["Pclass"] * int_df["Age"]
int_df["Age_fare"] = int_df["Age"] + int_df["Fare_per_person"]

In [283]:
df = pd.concat([int_df,str_df],axis=1)
df = df.astype('float')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 22 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Pclass           1309 non-null   float64
 1   Age              1309 non-null   float64
 2   SibSp            1309 non-null   float64
 3   Parch            1309 non-null   float64
 4   Fare             1309 non-null   float64
 5   family member    1309 non-null   float64
 6   Fare_per_person  1309 non-null   float64
 7   Cabin            1309 non-null   float64
 8   Pclass_age       1309 non-null   float64
 9   Age_fare         1309 non-null   float64
 10  Name__Dr         1309 non-null   float64
 11  Name__Major      1309 non-null   float64
 12  Name__Master     1309 non-null   float64
 13  Name__Miss       1309 non-null   float64
 14  Name__Mr         1309 non-null   float64
 15  Name__Mrs        1309 non-null   float64
 16  Name__Rev        1309 non-null   float64
 17  Sex__female    

In [284]:
train_data.shape

(891, 11)

In [285]:
X_train = df.iloc[:891,:].values
X_test = df.iloc[891:,:].values

In [286]:
y_train = pd.DataFrame(y_train)

In [287]:
y_train = y_train.iloc[:,-1].values

In [288]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [289]:
print(X_train.shape)
print(X_test.shape)
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters = 8)
y_kmeans_train = kmeans.fit_predict(X_train)
y_kmeans_test = kmeans.fit_predict(X_test)
y_kmeans_train = y_kmeans_train.reshape((len(y_kmeans_train),1))
y_kmeans_test = y_kmeans_test.reshape((len(y_kmeans_test),1))
print(y_kmeans_train.shape)
print(y_kmeans_test.shape)

(891, 22)
(418, 22)
(891, 1)
(418, 1)


In [290]:
print(X_train.shape)
print(X_test.shape)
X_train = np.concatenate((X_train,y_kmeans_train),axis=1)
X_test = np.concatenate((X_test,y_kmeans_test),axis=1)
print(X_train)

(891, 22)
(418, 22)
[[ 0.82737724 -0.42475618  0.43279337 ... -0.30756234  0.61930636
   1.        ]
 [-1.56610693  0.63091399  0.43279337 ... -0.30756234 -1.61470971
   2.        ]
 [ 0.82737724 -0.42475618 -0.4745452  ... -0.30756234  0.61930636
   3.        ]
 ...
 [ 0.82737724  0.1030789   0.43279337 ... -0.30756234  0.61930636
   3.        ]
 [-1.56610693 -0.42475618 -0.4745452  ... -0.30756234 -1.61470971
   4.        ]
 [ 0.82737724  0.1030789  -0.4745452  ...  3.25137334 -1.61470971
   1.        ]]


In [291]:
print(X_train.shape)
print(X_test.shape)

(891, 23)
(418, 23)


In [292]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 6)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

In [293]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
classifier.fit(X_train, y_train)

LogisticRegression(random_state=0)

In [294]:
y_pred = classifier.predict(X_test)

In [295]:
pred = pd.DataFrame(y_pred)
sub_df = pd.read_csv('gender_submission.csv')
dataset = pd.concat([sub_df['PassengerId'],pred],axis=1)
dataset.columns = sub_df.columns
dataset.to_csv('gender_submission.csv',index=False)

In [296]:
y_pred

array([0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1,
       1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1,
       1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1,
       0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,